# Natural Computing

This is an exercise from 2019/2 period of the course "Natural Computing" at UFES, in Vitória/ES - Brazil.

Task: Compare different nature-inspired optimization algorithms.

Master's degree candidate: Bruno Carvalho

## Algorithms

- GA - Genetic Algorithms
- PSO - Particle Swarm Optimization
- ES - Evolution Strategy


## References

.

In [1]:
import os
import sys
import time
import itertools
import random
import platform
import subprocess
import re

import psutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
from matplotlib import pyplot as plt

from benchmarks import *

%matplotlib inline
%config InlineBackend.figure_format = 'png'


In [ ]:
%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

# System

In [5]:
def get_processor_name():
    if platform.system() == "Windows":
        return platform.processor()
    elif platform.system() == "Darwin":
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin'
        command ="sysctl -n machdep.cpu.brand_string"
        return subprocess.check_output(command).strip()
    elif platform.system() == "Linux":
        command = "cat /proc/cpuinfo"
        all_info = subprocess.check_output(command, shell=True).decode('utf-8').strip()
        for line in all_info.split("\n"):
            if "model name" in line:
                return re.sub( ".*model name.*:", "", line,1)
    return ""

In [6]:
mem = psutil.virtual_memory()
swap = psutil.swap_memory()
print('OS', os.name)
print('system', platform.system(), platform.release())
print('cpu', platform.processor())
print('processors', psutil.cpu_count(), get_processor_name())
print('memory', f'{(mem.total / 1024 / 1024 / 1024):4.1f} GB')
print('python', platform.python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('matplotlib', mpl.__version__)

OS posix
system Linux 5.0.0-27-generic
cpu x86_64
processors 6  Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz
memory 31.3 GB
python 3.6.8
numpy 1.17.2
pandas 0.25.1
matplotlib 3.1.1


# Default parameters

In [3]:
#benchmarks = ['ackley_function', 'sphere_function', 'rastrigin_function', 'rosenbrock_function', 'schwefel']
benchmarks = ['ackley_function', 'sphere_function']

py_globals = globals()

execution_dict = {
    'time_elapsed': 0,
    'algorithm': '',
    'metrics': {
        'best': [],
        'all_best': 0.0,
        'all_worst': 0.0,
        'time': [],
        'objective': [],
        'scores': [],
    },
    'n_eval': 0,
    'max_eval': 0,
}

compare_cols = [
    'algorithm',
    'benchmark',
    'seed',
    'population',
    'dimension',
    'max_iter',
    'upper_bound',
    'lower_bound',
    'best',
    'worst',
    'mean',
    'time',
    'n_func_eval'
]
compare_data = pd.DataFrame(columns=compare_cols)

In [4]:
def ga_optimizer(obj_func, parameters):
    """

    Inspired by:
    https://github.com/7ossam81/EvoloPy/blob/master/GA.py

    """
    lower = parameters.get('lower', [-1.0])
    upper = parameters.get('upper', [1.0])
    dimension = parameters.get('dimension', 30)
    seed = parameters.get('seed', 1)
    max_iterations = parameters.get('max_iterations', 100)
    population = parameters.get('population', 100)
    crossover_rate = parameters.get('crossover_rate', 1.0)
    mutation_rate = parameters.get('mutation_rate', 0.01)
    best_to_keep = parameters.get('best_to_keep', 2)
    crossover_alpha = parameters.get('crossover_alpha', 0.25)
    max_eval = parameters.get('max_eval', 0)
    
    np.random.seed(seed)
    
    # Input Objective function has variable dimensions
    # consider equi-distance "square"
    if not isinstance(lower, list):
        lower = [lower]
    if not isinstance(upper, list):
        upper = [upper]

    lower = lower * dimension
    upper = upper * dimension

    # track execution
    exec_info = execution_dict.copy()
    n_eval = 0
    
    # allocate memory
    exec_info['metrics']['time'] = np.zeros([max_iterations], dtype='float')
    exec_info['metrics']['best'] = np.zeros([max_iterations, dimension], dtype='float')
    exec_info['metrics']['objective'] = np.zeros([max_iterations], dtype='float')
    exec_info['metrics']['scores'] = np.full([max_iterations, population], np.inf, dtype='float')

    begin = time.time()

    space = np.random.uniform(0, 1, [population, dimension])
    scores = np.random.uniform(0.0, 1.0, population)
    
    def crossover(space, scores):
        search_space = np.zeros_like(space) + np.random.uniform(0, 1)
        #search_space = np.zeros_like(space)
        search_space[0:best_to_keep, :] = space[0:best_to_keep, :]
        
        # renew BETA every generation
        beta = np.random.uniform(-crossover_alpha, 1 + crossover_alpha)
        
        for cross  in range(best_to_keep + 1, population - 1, 2):

            # using SET parents will always be different
            #parents = set()
            
            # using LIST parents can repeat!
            parents = []
            
            parent1_idx = np.random.randint(0, population)
            parent2_idx = np.random.randint(0, population)
            
            parents.append(parent1_idx)
            parents.append(parent2_idx)

            """
            while len(parents) < 2:
                parent1_idx = np.random.randint(0, population)
                parent2_idx = np.random.randint(0, population)
            
                if scores[parent1_idx] > scores[parent2_idx]:
                    # p1 is better then p2
                    parents.add(parent1_idx)
                else:
                    parents.add(parent2_idx)               
            """
            
            lp = list(parents)
            parent1 = space[lp[0], :]
            parent2 = space[lp[1], :]
            
            # ARITHMETIC CROSSOVER
            child1 = parent1 * beta + (1.0 - beta) * parent2
            child2 = parent2 * beta + (1.0 - beta) * parent1
            
            crossover_chance = np.random.uniform(0.0, 1.0)
            if crossover_chance < crossover_rate:
                search_space[cross, :] = np.copy(child1)
                search_space[cross + 1, :] = np.copy(child2)  
            else:
                search_space[cross, :] = np.copy(parent1)
                search_space[cross + 1, :] = np.copy(parent2)
            
            for j in range(dimension):
                search_space[cross, j] = np.clip(search_space[cross, j], lower[j], upper[j])
                search_space[cross + 1, j] = np.clip(search_space[cross + 1, j], lower[j], upper[j])
                
        return search_space
    
    def mutation(space, gen):
        n_mutate = np.int(population * mutation_rate)
        for m in range(n_mutate):
            # keep best => do not mutate
            rand_individual = np.random.randint(best_to_keep + 1, population)
            #rand_individual = np.random.randint(0, population)
            # decrease stdev with generations
            #stdev = 5.0 / np.sqrt(gen)
            stdev = 2
            new_value = np.zeros(dimension)
            for j in range(dimension):
                new_value[j] = np.random.normal(space[rand_individual, j], stdev)
                new_value[j] = np.clip(new_value[j], lower[j], upper[j])

            # store
            space[rand_individual, :] = new_value

        return space
    
    def sort_iter(_space, _scores):
        idx = scores.argsort()
        _space = _space[idx]
        _scores = _scores[idx]
        return _space, _scores
    
    def eval_obj(func, _space):
        _scores = np.full(population, np.inf)
        for p in range(population):
            _scores[p] = func(_space[p, :])
        return _scores

    for i in range(dimension):
        # init search space inside bounds
        space[:, i] = np.random.uniform(0, 1, population) * (upper[i] - lower[i]) + lower[i]
    
    for _iter in range(1, max_iterations):
        
        # crossover
        space = crossover(space, scores)
        
        # mutation
        space = mutation(space, _iter)
        
        # evaluate objective
        scores = eval_obj(obj_func, space)
        n_eval += population
        
        # remove duplicates
        
        # sort
        space, scores = sort_iter(space, scores)
        
        # save
        exec_info['metrics']['scores'][_iter] = scores
        exec_info['metrics']['time'][_iter] = time.time() - begin
        exec_info['metrics']['best'][_iter] = space[0, :]
        
        if _iter % np.int((max_iterations + 1) / 5) == 0:
            print(f'\t\tGen {_iter:06d}, '
                  f'f_min = {scores[0]:+13.5e}, '
            )

        if max_eval > 0 and _iter > max_eval:
            break

    exec_info['algorithm'] = 'GA - Genetic Algorithm'
    exec_info['time_elapsed'] = time.time() - begin
    exec_info['n_eval'] = n_eval
    
    return exec_info

In [5]:
def pso_optimizer(obj_func, parameters):
    """

    Inspired by:
    https://github.com/7ossam81/EvoloPy/blob/master/PSO.py

    """
    lower = parameters.get('lower', [-1.0])
    upper = parameters.get('upper', [1.0])
    dimension = parameters.get('dimension', 30)
    population = parameters.get('population', 100)
    seed = parameters.get('seed', 1)
    max_iterations = parameters.get('max_iterations', 100)
    velocity_max = np.full(dimension, parameters.get('velocity_max', 5.0))
    
    w_max = parameters.get('w_max', 0.9)
    w_min = parameters.get('w_min', 0.2)

    if not isinstance(lower, list):
        lower = [lower]
    if not isinstance(upper, list):
        upper = [upper]

    lower = lower * dimension
    upper = upper * dimension

    # track execution
    exec_info = execution_dict.copy()
    exec_info['metrics']['scores'] = np.zeros(max_iterations, dtype='float')

    # inner variables
    
    c1 = 2.0
    c2 = 2.0
    
    velocity = np.full([population, dimension], 0.0)
    
    p_best_score = np.full(population, np.inf)
    p_best = np.full([population, dimension], 0.0)
    
    g_best_score = np.inf
    g_best = np.full(population, 0.0)
    
    space = np.full([population, dimension], 0.0)
    
    begin = time.time()

    # init fill
    for _d in range(dimension):
        rand_dim = np.random.uniform(0, 1, population)
        space[:, _d] = rand_dim * (upper[_d] - lower[_d]) + lower[_d]
    
    n_eval = 0
    for _iter in range(max_iterations):
        for _p in range(population):
            # keep inside bounds / search limits
            space[_p, :] = np.clip(space[_p, :], lower, upper)
            
            # eval
            score = obj_func(space[_p, :])
            n_eval += 1
            
            # check
            if p_best_score[_p] > score:
                p_best_score[_p] = score
                p_best[_p, :] = np.copy(space[_p, :])
            
            if g_best_score > score:
                g_best_score = score
                g_best = np.copy(space[_p, :])
        
        w = w_max - _iter * ((w_max - w_min) / max_iterations)
        
        for _p in range(population):
            r1 = np.random.random([dimension])
            r2 = np.random.random([dimension])
            
            v = velocity[_p, :]
            
            a1 = c1 * r1 * (p_best[_p, :] - space[_p, :])
            a2 = c2 * r2 * (g_best - space[_p, :])
            
            velocity[_p, :] = w * v + a1 + a2
            
            velocity[_p, :] = np.fmax(velocity[_p, :], -velocity_max)  # negative!!
            velocity[_p, :] = np.fmin(velocity[_p, :],  velocity_max)
            
            # UPDATE space
            space[_p, :] += velocity[_p, :]

        # save data to analyse and plot later
        exec_info['metrics']['scores'][_iter] = g_best_score
        
        # some logging
        if _iter % np.int((max_iterations + 1) / 5.0) == 0:
            print(f'\t\tIteration {_iter:05d}, f_minx = {g_best_score:+12.5e}')

    # finished
    exec_info['algorithm'] = 'PSO - Particle Swarm Optimization'
    exec_info['time_elapsed'] = time.time() - begin
    exec_info['n_eval'] = n_eval

    return exec_info

In [6]:
def bb_pso_optimizer(obj_func, parameters):
    """


    """
    lower = parameters.get('lower', [-1.0])
    upper = parameters.get('upper', [1.0])
    dimension = parameters.get('dimension', 30)
    population = parameters.get('population', 100)
    seed = parameters.get('seed', 1)
    max_iterations = parameters.get('max_iterations', 100)
    velocity_max = np.full(dimension, parameters.get('velocity_max', 5.0))

    np.random.seed(seed)

    w_max = parameters.get('w_max', 0.9)
    w_min = parameters.get('w_min', 0.2)

    if not isinstance(lower, list):
        lower = [lower]
    if not isinstance(upper, list):
        upper = [upper]

    lower = lower * dimension
    upper = upper * dimension

    # track execution
    exec_info = execution_dict.copy()
    exec_info['metrics']['scores'] = np.zeros(max_iterations, dtype='float')

    # inner variables

    c1 = 2.0
    c2 = 2.0

    velocity = np.full([population, dimension], 0.0)

    p_best_score = np.full(population, np.inf)
    p_best = np.full([population, dimension], 0.0)

    g_best_score = np.inf
    g_best = np.full(population, 0.0)

    space = np.full([population, dimension], 0.0)

    begin = time.time()

    # init fill
    for _d in range(dimension):
        rand_dim = np.random.uniform(0, 1, population)
        space[:, _d] = rand_dim * (upper[_d] - lower[_d]) + lower[_d]

    n_eval = 0
    for _iter in range(max_iterations):
        for _p in range(population):
            # keep inside bounds / search limits
            space[_p, :] = np.clip(space[_p, :], lower, upper)

            # eval
            score = obj_func(space[_p, :])
            n_eval += 1

            # check
            if p_best_score[_p] > score:
                p_best_score[_p] = score
                p_best[_p, :] = np.copy(space[_p, :])

            if g_best_score > score:
                g_best_score = score
                g_best = np.copy(space[_p, :])

        w = w_max - _iter * ((w_max - w_min) / max_iterations)

        for _p in range(population):
            r1 = np.random.random([dimension])
            r2 = np.random.random([dimension])

            v = velocity[_p, :]

            a1 = c1 * r1 * (p_best[_p, :] - space[_p, :])
            a2 = c2 * r2 * (g_best - space[_p, :])

            velocity[_p, :] = w * v + a1 + a2

            velocity[_p, :] = np.fmax(velocity[_p, :], -velocity_max)  # negative!!
            velocity[_p, :] = np.fmin(velocity[_p, :],  velocity_max)

            # UPDATE space
            space[_p, :] += velocity[_p, :]

        # save data to analyse and plot later
        exec_info['metrics']['scores'][_iter] = g_best_score

        # some logging
        if _iter % np.int(max_iterations / 10.0) == 0:
            print(f'\tIteration {_iter:05d}, f_minx = {g_best_score:+12.5e}')

    # finished
    exec_info['algorithm'] = 'BBPSO'
    exec_info['time_elapsed'] = time.time() - begin

    return exec_info

In [7]:
def es_optimizer(obj_func, parameters):
    """
    """
    seed = parameters.get('seed', 1)
    lower = parameters.get('lower', [-1.0])
    upper = parameters.get('upper', [1.0])
    dimension = parameters.get('dimension', 30)
    population = parameters.get('population', 100)
    max_iterations = parameters.get('max_iterations', 100)
    crossover_rate = parameters.get('crossover_rate', 1.0)
    mutation_rate = parameters.get('mutation_rate', 0.01)
    
    np.random.seed(seed)
    begin = time.time()
    n_eval = 0
    exec_info['algorithm'] = 'ES'
    exec_info['time_elapsed'] = time.time() - begin
    exec_info['n_eval'] = n_eval

In [8]:
def cma_es_optimizer(obj_func, parameters):
    """
    """
    seed = parameters.get('seed', 1)
    lower = parameters.get('lower', [-1.0])
    upper = parameters.get('upper', [1.0])
    dimension = parameters.get('dimension', 30)
    population = parameters.get('population', 100)
    max_iterations = parameters.get('max_iterations', 100)
    crossover_rate = parameters.get('crossover_rate', 1.0)
    mutation_rate = parameters.get('mutation_rate', 0.01)

    
    np.random.seed(seed)
    begin = time.time()
    n_eval = 0
    exec_info['algorithm'] = 'CMA-ES'
    exec_info['time_elapsed'] = time.time() - begin
    exec_info['n_eval'] = n_eval

# GA - Genetic Algorithm

In [9]:
results = []
_dim = 30

for bmrk in benchmarks:
    print('Benchmark', bmrk)
    for seed in range(1, 2):
        params = {
            'seed': seed,
            'dimension': _dim,
            'max_iterations': 1000,
            'population': 100,
            'upper': [30],
            'lower': [-30],
        }
        print(f'\tDimensions: {_dim}, Seed={seed}')
        tmp_result = ga_optimizer(py_globals[bmrk], params)
        results.append(tmp_result.copy())
        print(f"\t\tTop 3 f(x) values, after {tmp_result['time_elapsed']:7.2f}s")
        print('\t\tf=', tmp_result['metrics']['scores'][-1][:3])
        #print('\tx=')
        #for xv in tmp_result['metrics']['best'][-1]:
        #    print(f'\t\t {xv:+12.7e}')
        #    pass

        compare_data = compare_data.append(
            pd.DataFrame(data=[[
                'GA',
                bmrk,
                seed,
                params['population'],
                _dim,
                params['max_iterations'],
                params['upper'][0],
                params['lower'][0],
                np.amin(tmp_result['metrics']['scores'][-1]),
                np.inf,
                np.inf,
                tmp_result['time_elapsed'],
                tmp_result['n_eval']
            ]], columns=compare_cols),
            ignore_index=True
        )

Benchmark ackley_function
	Dimensions: 30, Seed=1
		Gen 000200, f_min =  +6.36001e-03, 
		Gen 000400, f_min =  +3.18091e-04, 
		Gen 000600, f_min =  +3.18091e-04, 
		Gen 000800, f_min =  +3.18091e-04, 
		Top 3 f(x) values, after   34.28s
		f= [0.00031809 0.00031809 0.18064996]
Benchmark sphere_function
	Dimensions: 30, Seed=1
		Gen 000200, f_min =  +3.92370e-04, 
		Gen 000400, f_min =  +1.89315e-07, 
		Gen 000600, f_min =  +1.89315e-07, 
		Gen 000800, f_min =  +1.89315e-07, 
		Top 3 f(x) values, after   34.16s
		f= [1.89314808e-07 1.89314808e-07 3.47494405e-02]


In [ ]:
r_dim = results[0]
y_plot = []
for _iter2 in range(r_dim['metrics']['scores'].shape[0]):
    y_plot.append(r_dim['metrics']['scores'][_iter2][0])

In [ ]:
# change plot scale to LOG => better view
plt.plot(-np.log(y_plot))

plt.title('GA - Genetic Algorithm')
#plt.subtitle('log of f(x) for x = best')
plt.xlabel('generations')
plt.ylabel('log of f')

In [ ]:
pt = pd.pivot_table(
    compare_data[compare_data['algorithm'] == 'GA'],
    values=['best', 'worst', 'time', 'n_func_eval'],
    #values=['best'],
    index=['benchmark'],
    aggfunc={
        'best': [np.amin, np.amax],
        'worst': np.amax,
        'time': np.mean,
        'n_func_eval': np.amax
    }
)
print(pt)

# PSO - Particle Swarm Optimization algorithm

In [ ]:
results = []
_dim = 30

for bmrk in benchmarks:
    print('Benchmark function', bmrk)
    for seed in range(1, 2):
        params = {
            'seed': seed,
            'dimension': _dim,
            'max_iterations': 1000,
            'population': 100,
            'upper': [30],
            'lower': [-30],
        }
        print(f'\tDimensions: {_dim}, Seed={seed}')
        tmp_result = pso_optimizer(ackley_function, params)
        results.append(tmp_result.copy())
        print(f"\t\tTop 3 f(x) values, after {tmp_result['time_elapsed']:7.2f}s")
        print('\t\tf=', tmp_result['metrics']['scores'][-3:])

        compare_data = compare_data.append(
            pd.DataFrame(data=[[
                'PSO',
                bmrk,
                seed,
                params['population'],
                _dim,
                params['max_iterations'],
                params['upper'][0],
                params['lower'][0],
                np.amin(tmp_result['metrics']['scores'][-1]),
                np.inf,
                np.inf,
                tmp_result['time_elapsed'],
                tmp_result['n_eval']
            ]], columns=compare_cols),
            ignore_index=True
        )

In [ ]:
r_dim = results[0]
y_plot = []
for _iter2 in range(r_dim['metrics']['scores'].shape[0]):
    y_plot.append(r_dim['metrics']['scores'][_iter2])

In [ ]:
plt.plot(-np.log(y_plot))
plt.title('PSO - Particle Swarm Optimization')
#plt.subtitle('log of f(x) for x = best')
plt.xlabel('generations')
plt.ylabel('log of f')

## BBPSO - Bare-bone Particle Swarm Optimization algorithm

In [ ]:
results = []
_dim = 30

for seed in range(1, 5):
    params = {
        'seed': seed,
        'dimension': _dim,
        'max_iterations': 1000,
        'population': 100,
        'upper': [30],
        'lower': [-30],
    }
    print(f'Dimensions: {_dim}')
    tmp_result = bb_pso_optimizer(ackley_function, params)
    results.append(tmp_result.copy())
    print(f"\tTop 5 f(x) values, after {tmp_result['time_elapsed']:7.2f}s")
    print('\tf=', tmp_result['metrics']['scores'][-5:])


# ES - Evolutionary Strategy algorithm

In [ ]:
0

# CMA-ES - Covariance Matrix Evolutionary Strategy algorithm

# Direct comparison

Compare results for each benchmark function.

In [ ]:
pt = pd.pivot_table(
    compare_data,
    #values=['best', 'worst', 'time', 'n_func_eval'],
    values=['best', 'time', 'n_func_eval'],
    index=['benchmark', 'algorithm'],
    aggfunc={
        'best': [np.amin, np.amax],
        #'worst': np.amax,
        'time': np.mean,
        'n_func_eval': np.amax
    }
)
print(pt)

In [ ]:
print(type(pt))

In [ ]:
print(pt.columns)

In [ ]:
colors = {
    'ackley_function': 'b',
    'sphere_function': 'r'
}
markers = {
    'GA': 's',
    'PSO': 'o'
}

for i in range(pt.shape[0]):
    f = pt.iloc[i, pt.columns.gel_loc('benchmark')]
    a = pt.iloc[i, pt.gel_loc('algorithm')]
    plot(pt.iloc[i, pt.gel_loc('time')],
         pt.iloc[i, pt.gel_loc('amax')],
         color=colors[f], marker=markers[a], linestyle='none', markersize=12)